In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)
df

,artist,song,text
0,Thin Lizzy,Got To Give It Up,[Chorus] \r\nI've got to give it up I've got ...
1,Ray Boltz,All You Died For Me To Be,All You Died For Me To Be \r\nWords and music...
2,Reo Speedwagon,Good Trouble,"Babe, I'm gettin' the message it's finally com..."
3,Bonnie Raitt,Love Letter,"Sittin' in front of your house, \r\nLike rain..."
4,Housemartins,Lean On Me,Heavy burden on your shoulder \r\nLean on me ...
...,...,...,...
4995,Underworld,Bruce Lee,(bullet got the wrong bloke) \r\n \r\nLife k...
4996,Counting Crows,God Of Ocean Tides,Close the door on a short night \r\nLift the ...
4997,Keith Green,Stained Glass,We are like windows \r\nStained with colors o...
4998,Bette Midler,Whose Side Are You On?,Every time I touch you \r\nMy heart skips a b...


Text Cleaning / Text Preprocessing

In [6]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [7]:
import nltk
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Microsoft\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)
stemmer = PorterStemmer()

In [44]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [46]:
df['text']=df['text'].apply(lambda x: tokenization(x))

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [15]:
matrix

<5000x24088 sparse matrix of type '<class 'numpy.float64'>'
	with 266787 stored elements in Compressed Sparse Row format>

In [61]:
similarity = cosine_similarity(matrix)

In [60]:
df[df['song'] == 'Got To Give It Up']

,artist,song,text
0,Thin Lizzy,Got To Give It Up,[ choru ] i 've got to give it up i 've got to...


In [67]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:11]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [68]:
recommendation("Good Trouble")

['I Smell Trouble',
 'Casey Jones',
 'Woman Trouble Blues',
 "You've Got A Friend",
 'Working On The Road',
 "Don't Know Why",
 'Good Life',
 'Still She Cries',
 'Let Me Entertain You',
 'Oh How The Years Go By']